# LangChain 实战：家電销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的家電产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在家電销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国家電产销售的实用话术：
源可以选择。

1. [客户问题]：这个电视有什么特点？[销售回答]：这款电视采用了最新的4K技术，能够提供更清晰的画面和更广泛的色彩范围。
2. [客户问题]：这款冰箱的容量大吗？[销售回答]：是的，这款冰箱的容量非常大，适合大户型家庭使用。
3. [客户问题]：这个空调的能耗高吗？[销售回答]：不，这款空调采用了节能技术，能够有效降低能耗。
4. [客户问题]：这款洗衣机的噪音大吗？[销售回答]：不，这款洗衣机采用了静音设计，运行非常安静。
5. [客户问题]：这个微波炉安全吗？[销售回答]：是的，这款微波炉采用了多重安全保护，确保使用安全。
6. [客户问题]：这款电饭煲好用吗？[销售回答]：是的，这款电饭煲采用了智能控制，能够自动调节温度和时间。
7. [客户问题]：这个咖啡机怎么用？[销售回答]：这款咖啡机操作非常简单，只需添加咖啡豆和水就可以制作出美味的咖啡。
8. [客户问题]：这款电烤箱的温度控制准确吗？[销售回答]：是的，这款电烤箱采用了精确的温度控制系统，能够确保食物均匀烹饪。
9. [客户问题]：这个吸尘器的吸力强吗？[销售回答]：是的，这款吸尘器采用了强大的吸力，能够有效清洁地板和地毯。
10. [客户问题]：这款电风扇的噪音大吗？[销售回答]：不，这款电风扇采用了静音设计，运行非常安静。

## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [ ]:
with open("sales.txt") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
text_splitter = CharacterTextSplitter(
    separator = r'\d+\.',  # Matches any number followed by a dot
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [ ]:
docs = text_splitter.create_documents([real_estate_sales])

In [ ]:
docs[0]

Document(page_content='\\d+\\. [客户问题]：这个电视有什么特点？\n\u2028[销售回答]：这款电视采用了最新的4K技术，能够提供更清晰的画面和更广泛的色彩范围。')

In [ ]:
len(docs)

98

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [ ]:
from langchain_mistralai import MistralAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs,  MistralAIEmbeddings(api_key="rDjmdXGVUS37qMW2NOFP5vCpqlpve6Yt"))

/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [ ]:
query = "这个微波炉安全吗"

In [ ]:
answer_list = db.similarity_search(query)

In [ ]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]：这个微波炉安全吗？
 [销售回答]：是的，这款微波炉采用了多重安全保护，确保使用安全。

[客户问题]：这个电水壶安全吗？
 [销售回答]：是的，这款电水壶采用了多重安全保护，确保使用安全。

[客户问题]：这个电水壶的安全性好吗？
 [销售回答]：是的，这款电水壶采用了多重安全保护，确保使用安全。

[客户问题]：这个电水壶的安全性好吗？
 [销售回答]：是的，这款电水壶采用了多重安全保护，确保使用安全。



In [ ]:
db.save_local("real_machine_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [ ]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [ ]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'MistralAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7ee053f39ba0>, search_kwargs={'k': 3})

In [ ]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：这个微波炉安全吗？
 [销售回答]：是的，这款微波炉采用了多重安全保护，确保使用安全。

[客户问题]：这个电水壶安全吗？
 [销售回答]：是的，这款电水壶采用了多重安全保护，确保使用安全。

[客户问题]：这个电水壶的安全性好吗？
 [销售回答]：是的，这款电水壶采用了多重安全保护，确保使用安全。



In [ ]:
docs = topK_retriever.get_relevant_documents("你们有没有风扇？")

In [ ]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：这款电风扇的能耗高吗？
 [销售回答]：不，这款电风扇采用了节能技术，能够有效降低能耗。

[客户问题]：这款电风扇的能耗高吗？
 [销售回答]：不，这款电风扇采用了节能技术，能够有效降低能耗。

[客户问题]：这款电风扇的能耗高吗？
 [销售回答]：不，这款电风扇采用了节能技术，能够有效降低能耗。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [ ]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [ ]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：这个微波炉安全吗？
 [销售回答]：是的，这款微波炉采用了多重安全保护，确保使用安全。

